In [5]:
import sympy as sp
import numpy as np
import time
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import numpy.random as rnd
from math import ceil

In [2]:
class Symbols:

    def __init__(self):
        # initialize
        self.reference = []
        self.values = []
        # initialize map from string to int and back
        self.reference2id = {}
        self.names2id = {}
        self.id2reference = {}
        self.dimension = 0

    #Adds a new symbol to the symbol array, creating a sympy object
    def add(self, name, value):
        symbol = sp.symbols(name)
        index = len(self.reference)
        self.reference.append(symbol)
        self.values.append(value)
        self.reference2id[symbol] = index
        self.names2id[name] = index
        self.id2reference[index] = symbol
        self.dimension += 1
        return symbol

    # sets the value of a symbol
    def set(self, name, value):
        try:
            index = self.names2id[name]
            self.values[index] = value
        except:
            print("Symbol " + name + " is not defined")

    def get_value(self,name):
        try:
            index = self.names2id[name]
            return self.values[index]
        except:
            print("Symbol " + name + " is not defined")

    #finalizes the symbol array, generating a numpy array for values
    def finalize(self):
        self.values = np.array(self.values)

    def get_id(self, name):
        return self.names2id[name]

In [3]:
class Transition:

    def __init__(self, update, rate, sympy_symbols):
        #this will convert rate into a sympy expression.
        #we need a sanity check before?
        self.update = None
        self.update_dictionary = update
        try:
            self.rate = sp.sympify(rate, sympy_symbols, evaluate=False)
        except sp.SympifyError as e:
            print("An error happened while parsing expression", rate,":",e)

    # finalizes transition by turning the update list into a numpy array
    def finalize(self, variables):
        self.update = np.zeros(variables.dimension)
        for var_name in self.update_dictionary:
            index = variables.get_id(var_name)
            self.update[index] = self.update_dictionary[var_name]
        self.update = np.reshape(self.update, (1,variables.dimension))

In [4]:
class Observable:

    def __init__(self, observable_function, sympy_symbols):
        try:
            self.function = sp.sympify(observable_function, sympy_symbols, evaluate=False)
        except sp.SympifyError as e:
            print("An error happened while parsing expression", observable_function, ":", e)

    # finalizes transition by turning the update list into a numpy array
    def finalize(self, variables):
        n = variables.dimension
        x = variables.reference
        self.gradient = np.zeros((1, n), dtype=object)
        self.hessian = np.zeros((n, n), dtype=object)
        for i in range(n):
            self.gradient[0,i] = sp.diff(self.function, x[i])
            for j in range(i,n):
                self.hessian[i, j] = sp.diff(self.function, x[i], x[j])
                self.hessian[j, i] = self.hessian[i, j]

#### Syntax to add transitions:

pctmc = Model()

pctmc.add_transition({"A": -1, "B": +1}, "rate*A")

In [5]:
class Model:

    def __init__(self):
        #init variables and parameters
        self.variables = Symbols()
        self.parameters = Symbols()
        # this contains a map of names to sympy variables, to be used later for parsing expressions
        self.names2sym = {}
        # init transition list
        self.transitions = []
        self.transition_number = 0
        self.observable_list = []
        self.observable_dimension = 0
        self.system_size = 0;
        self.system_size_reference = None
        self.system_size_name = ''
        self.observable_names = []

    def set_system_size(self, name, value):
        self.add_parameter(name, value)
        self.system_size_reference = self.names2sym[name]
        self.system_size_name = name
        self.system_size = value


    def add_variable(self, name, value):
        if name in self.names2sym:
            raise ModelError("Name " + name + " already defined!")
        var = self.variables.add(name, value)
        self.names2sym[name] = var

    def add_parameter(self, name, value):
        if name in self.names2sym:
            raise ModelError("Name " + name + " already defined!")
        par = self.parameters.add(name, value)
        self.names2sym[name] = par

    # Changes the initial value of a variable
    def set_variable(self, name, value):
        self.variables.set(name, value)

    # Changes the value of a parameter
    def set_parameter(self, name, value):
        if self.system_size_name == name:
            self.parameters.set(name, value)
            self.system_size = value
        else:
            self.parameters.set(name, value)

    def get_parameter_value(self, name):
        return self.parameters.get_value(name)

    # Adds a transition to the model
    def add_transition(self, update, rate):
        t = Transition(update, rate, self.names2sym)
        self.transitions.append(t)
        self.transition_number += 1

    def add_observable(self, obs_name, observable):
        obs = Observable(observable, self.names2sym)
        self.observable_list.append(obs)
        self.observable_names.append(obs_name)
        self.observable_dimension += 1

    # Finalizes the initialization
    def finalize_initialization(self):
        self.variables.finalize()
        self.parameters.finalize()
        for t in self.transitions:
            t.finalize(self.variables)
        for obs in self.observable_list:
            obs.finalize(self.variables)

        self.__generate_observable_functions()
        self.__generate_numpy_functions()
        



    def __generate_observable_functions(self):
        n = self.variables.dimension
        p = len(self.observable_list)
        self._observables_sympy = np.zeros((1, p), dtype=object)


    # generate numpy expressions and the mean field VF
    def __generate_numpy_functions(self):
        sympy_ref = self.variables.reference + self.parameters.reference
        self.rates = sp.lambdify(sympy_ref, [t.rate for t in self.transitions], "numpy")
        


    # evaluates and returns observables and their jacobian and hessian ...
    def evaluate_observables(self, var_values):
        g = self.observables(*var_values, *self.parameters.values)
        return np.asarray(g)

    def evaluate_rates(self, var_values):
        r = self.rates(*var_values, *self.parameters.values)
        return np.asarray(r)


Class for plotting trajectories of your simulations

In [6]:
# class containing a trajectory,
class Trajectory:
    def __init__(self, t, x, desc, labels):
        self.time = t
        self.data = x
        self.labels = labels
        self.description = desc

    def plot(self, var_to_plot=None):
        if var_to_plot is None:
            var_to_plot = self.labels
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_prop_cycle(plt.cycler('color', ['r', 'g', 'b', 'c', 'm', 'y', 'k']))
        handles = []
        labels = []
        for v in var_to_plot:
            try:
                i = self.labels.index(v)
                h, = ax.plot(self.time, self.data[:, i])
                handles.append(h)
                labels.append(v)
            except:
                print("Variable", v, "not found")
        fig.legend(handles, labels)
        plt.title(self.description)
        plt.xlabel('Time')
        plt.show()